### A linear Poisson equation
$\nabla^2 u = sin(\pi x) sin(\pi y)$ \
$0 \leq x \leq 1$ \
$0 \leq y \leq 1$

In [ ]:
# partial differential equation
def f(u):
    out = torch.sin(torch.tensor(math.pi)*u[0]) * torch.sin(torch.tensor(math.pi)*u[1])
    return out, 2

# boundary conditions
def g(u):
    return 0, 0